#MARKET BASKET ANALYSIS USING APRIORI ALGORITHM
##By Elma Alifa Nadya
This project is possible thanks to MySkill E-Learning: Python for Data Analyst and the tutors Muhammad Ihza Mahendra & Ivan Adhitama Christanto.

#Pendahuluan

## Import Library

In [1]:
#import Library
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import datetime as dt

## Import dataframe

In [2]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1J8h9oVngI164ao6C2y46JhZK1Zjfh8Zc/edit#gid=1839236009'
sheet_url_trf = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=') # convert editable spreadsheet csv
print(sheet_url_trf)

df = pd.read_csv(sheet_url_trf) 
df.info()
df.head() 

https://docs.google.com/spreadsheets/d/1J8h9oVngI164ao6C2y46JhZK1Zjfh8Zc/export?format=csv&gid=1839236009
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


## Data cleansing

In [3]:
df_clean = df.copy()
# mengkonversi kolom order_date menjadi datetime
df_clean['order_date'] = df_clean['order_date'].astype('datetime64')
# membuat kolom date
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64')

# menghapus semua baris tanpa customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()
# menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# membuat kolom order_status dengan nilai 'cancelled' jika order_id diawali dengan huruf 'c' dan 'delivered' jika order_id tanpa awalan huruf 'c'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

<ipython-input-3-2f3079ed661f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['amount'] = df_clean['quantity'] * df_clean['price']


,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,cancelled,4.25
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,51.00
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,5.10
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,8.50
...,...,...,...,...,...,...,...,...,...,...
358464,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,delivered,1.25
358465,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358466,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358467,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,delivered,9.90


In [4]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358469 entries, 0 to 358468
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      358469 non-null  object        
 1   product_code  358469 non-null  object        
 2   product_name  358469 non-null  object        
 3   quantity      358469 non-null  int64         
 4   order_date    358469 non-null  datetime64[ns]
 5   price         358469 non-null  float64       
 6   customer_id   358469 non-null  object        
 7   date          358469 non-null  datetime64[ns]
 8   order_status  358469 non-null  object        
 9   amount        358469 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 27.3+ MB


#Bagian 1 - Menyiapkan data basket

##1.1 Buat DataFrame basket

In [5]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C539947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C539949,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C539950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, 493414 to C539983
Columns: 3862 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3862)
memory usage: 587.8+ MB


##1.2 Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0

In [7]:
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.applymap(encode)
basket_encode

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C539947,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C539949,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C539950,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, 493414 to C539983
Columns: 3862 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3862)
memory usage: 73.6+ MB


##1.3 Ambil transaksi dengan banyaknya produk unik lebih dari 1 saja

In [9]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C539711,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C539719,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C539943,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16613 entries, 493414 to C539950
Columns: 3862 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3862)
memory usage: 61.3+ MB


#Bagian 2 - Mengaplikasikan apriori algorithm

##2.1 Buat list frequent itemset (kumpulan produk yang sering dibeli)

In [20]:
# !pip install mlxtend

In [19]:
from mlxtend.frequent_patterns import apriori
frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

,support,itemsets,product_cnt
0,0.166075,(white hanging heart t-light holder),1
1,0.094865,(regency cakestand 3 tier),1
2,0.089930,(jumbo bag red retrospot),1
3,0.072895,(strawberry ceramic trinket box),1
4,0.072473,(pack of 72 retro spot cake cases),1
...,...,...,...
995,0.010052,"(white hanging heart t-light holder, red retro...",2
996,0.010052,"(white hanging heart t-light holder, metal 4 h...",2
997,0.010052,"(ivory kitchen scales, red kitchen scales)",2
998,0.010052,"(key fob , shed, key fob , front door )",2


##2.2 Hitung nilai support, confidence, dan lift dari setiap pasangan produk yang mungkin

In [17]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7).sort_values(['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.054174,0.166075,0.038885,0.717778,4.322016,0.029888,2.954853
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.046169,0.072895,0.034912,0.756193,10.373768,0.031547,3.802619
2,(toilet metal sign),(bathroom metal sign),0.024619,0.036839,0.019623,0.797066,21.636696,0.018716,4.746181
3,(painted metal pears assorted),(assorted colour bird ornament),0.020285,0.071570,0.015410,0.759644,10.613931,0.013958,3.862725
4,"(jumbo bag strawberry, jumbo bag pink with whi...",(jumbo bag red retrospot),0.020406,0.089930,0.014928,0.731563,8.134848,0.013093,3.390262
...,...,...,...,...,...,...,...,...,...
56,"(wooden picture frame white finish, wood s/3 c...","(wooden frame antique white, wood 2 drawer cab...",0.014326,0.016252,0.010414,0.726891,44.725319,0.010181,3.602030
57,"(feltcraft cushion owl, feltcraft cushion rabbit)",(feltcraft cushion butterfly),0.014146,0.024920,0.010353,0.731915,29.370295,0.010001,3.637202
58,"(feltcraft cushion owl, feltcraft cushion butt...",(feltcraft cushion rabbit),0.014386,0.023235,0.010353,0.719665,30.973573,0.010019,3.484282
59,"(strawberry charlotte bag, charlotte bag pink ...",(red spotty charlotte bag),0.013002,0.037079,0.010293,0.791667,21.350582,0.009811,4.622019
